In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor


/Users/Grace/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [43]:
"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")


In [44]:
df_train.head()


,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1.19
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1,0,0,0,1,0,1,0,0,...,1,0,0,1,0,0,0,0,0,1.60
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1,0,0,0,1,1,1,0,0,...,1,0,0,0,1,0,0,0,0,1.49
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1,0,0,0,1,1,1,0,0,...,1,0,0,0,1,0,0,0,0,1.36
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1.98


In [45]:
print df_train.shape

(1000000, 258)


In [46]:
df_test.head()


,Id,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,1,c1sc(-c2cnc3c(c2)c2nsnc2c2cc4cccnc4cc32)c2cc[n...,0,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
1,2,[nH]1cccc1-c1cc2c3nsnc3c3c4sccc4[nH]c3c2s1,0,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
2,3,[nH]1c2cc(-c3ccc[se]3)c3nsnc3c2c2c3cscc3c3ccc4...,1,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
3,4,[nH]1c(cc2cnc3c(c12)c1=C[SiH2]C=c1c1ccc2=CCC=c...,1,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
4,5,c1sc(-c2sc(-c3sc(-c4scc5[se]ccc45)c4ccoc34)c3c...,0,0,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [47]:
print df_test.shape

(824230, 258)


In [48]:
#store gap values
Y_train = df_train.gap.values
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column
df_test = df_test.drop(['Id'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)


In [49]:
test_idx

1000000

In [50]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_test), axis=0)
df_all.head()


,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1,0,0,0,1,0,1,0,0,...,0,1,0,0,1,0,0,0,0,0
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [51]:
df_all.shape

(1824230, 257)

In [52]:
print df_all.head(20).smiles.values

['c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2nsnc12'
 'C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[SiH2]C=c12'
 '[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-23)c2ccccc12'
 '[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13)c1=C[SiH2]C=c21'
 'c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1'
 'C1=Cc2cnc3cc4cc(-c5scc6[nH]ccc56)c5ccccc5c4cc3c2[SiH2]1'
 'c1ncc(s1)-c1cnc2c(c1)oc1c2ccc2ccccc12'
 'c1sc(-c2ccc3c(c2)sc2c3c3=CCC=c3c3cccnc23)c2[se]ccc12'
 'c1ccc(o1)-c1cc2cc3cc4c5c[nH]cc5ccc4cc3cc2o1'
 '[nH]1ccc2c3c[nH]cc3c3cc(-c4cncs4)c4=CCC=c4c3c12'
 '[nH]1c(cc2c3cocc3c3c(ccc4ccc5=CCC=c5c34)c12)-c1cccs1'
 'c1cc2oc3c(sc4cc([se]c34)-c3cncc4nsnc34)c2o1'
 '[nH]1c(cc2cnc3cc4ccoc4cc3c12)-c1ccccc1'
 '[nH]1ccc2ccc3c4ncc(cc4[nH]c3c12)-c1scc2occc12'
 'c1sc(-c2sc(-c3sc(-c4ncncn4)c4nccnc34)c3cc[nH]c23)c2ccCc12'
 'c1cc2ncc(cc2s1)-c1cc2c(ccc3ccccc23)c2c[nH]cc12'
 'c1ccc(-c2cc3oc4ccc5c[nH]cc5c4c3cn2)c2cscc12'
 '[nH]1c2-c3ncc(cc3Cc2c2[se]ccc12)-c1scc2ccoc12'
 'c1cnc(s1)-c1ccc(cc1)-c1sc(c2Cccc12)-c1scc2occc12'
 'c

In [53]:
"""
Example Feature Engineering

this calculates the length of each smile string and adds a feature column with those lengths
Note: this is NOT a good feature and will result in a lower score!
"""
#smiles_len = np.vstack(df_all.smiles.astype(str).apply(lambda x: len(x)))
#df_all['smiles_len'] = pd.DataFrame(smiles_len)

df_all['small_carbon'] = df_all['smiles'].apply(lambda r: r.count('c'))
df_all['big_carbon'] = df_all['smiles'].apply(lambda r: r.count('C'))
df_all['oxygen'] = df_all['smiles'].apply(lambda r: r.count('o'))
df_all['nitrogen'] = df_all['smiles'].apply(lambda r: r.count('n'))
df_all['sulfur'] = df_all['smiles'].apply(lambda r: r.count('s'))
df_all['hydrogen'] = df_all['smiles'].apply(lambda r: r.count('H'))
df_all['[]'] = df_all['smiles'].apply(lambda r: r.count('['))
df_all['()'] = df_all['smiles'].apply(lambda r: r.count('('))
# df_all['big_c']=df_all['smiles'].apply(lambda r: r.count('C'))
# df_all['nH']=df_all['smiles'].apply(lambda r: r.count('[nH]'))
# df_all['SiH2']=df_all['smiles'].apply(lambda r: r.count('[SiH2]'))
df_all['onelink']=df_all['smiles'].apply(lambda r: r.count('-'))
df_all['doublelink']=df_all['smiles'].apply(lambda r: r.count('='))
# df_all['subgroup']=df_all['smiles'].apply(lambda r: r.count('('))
# df_all['se']=df_all['smiles'].apply(lambda r: r.count('se'))
# df_all['doublelinkC']=df_all['smiles'].apply(lambda r: r.count('=C'))
# df_all['cnc']=df_all['smiles'].apply(lambda r: r.count('cnc'))


# featAd=['small_c','big_c','nH','SiH2','onelink','doublelink','subgroup','se','cnc','doublelinkC']

df_all.head()


,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,small_carbon,big_carbon,oxygen,nitrogen,sulfur,hydrogen,[],(),onelink,doublelink
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0,0,0,0,1,0,1,0,0,...,19,0,1,3,4,0,1,3,3,0
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1,0,0,0,1,0,1,0,0,...,12,9,0,2,0,2,2,1,0,5
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1,0,0,0,1,1,1,0,0,...,18,3,0,3,2,2,2,2,3,1
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1,0,0,0,1,1,1,0,0,...,18,5,1,1,0,3,3,1,2,4
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0,0,0,0,1,0,1,0,0,...,20,0,2,6,1,0,0,1,1,0


In [18]:
#Drop the 'smiles' column
df_all = df_all.drop(['smiles'], axis=1)
vals = df_all.values # [0, 1, 0, ..., 14, 6] values of each feature
X_train = vals[:test_idx] # [:, :] #cut test_idx in row direction
X_test = vals[test_idx:]
print "Train features:", X_train.shape
print "Train gap:", Y_train.shape
print "Test features:", X_test.shape


Train features: (1000000, 265)
Train gap: (1000000,)
Test features: (824230, 265)


In [54]:
Y_train

array([ 1.19,  1.6 ,  1.49, ...,  2.03,  1.74,  1.72])

# Utility fo submit to Kaggle

In [20]:
practical_small_data_size = 10000

In [24]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

# Base Model (with Chemical features)

## 1. Linear Regression

In [55]:
from sklearn.metrics import mean_squared_error

LR = LinearRegression()
LR.fit(X_train, Y_train)
print "The Linear Regression Score is %0.6f" %LR.score(X_train,Y_train)

LR_pred = LR.predict(X_train)
print "Train data MSE is ",mean_squared_error(Y_train,LR_pred)

# =========================
# originally we're to predict with X_test and submit Y_test
# =========================
# LR_pred = LR.predict(X_test)

The Linear Regression Score is 0.710504
Train data MSE is  0.0479962137863


In [23]:
print LR_pred.shape
print LR_pred

(1000000,)
[ 1.53191829  1.73054766  1.65184886 ...,  1.74493906  1.95732842
  1.61140821]


In [25]:
LR_pred_test = LR.predict(X_test)

In [26]:
LR_pred_test.shape

(824230,)

In [27]:
write_to_file("LR_Grace_chemical.csv", LR_pred_test)

## 2. Random Forest Regressor

In [56]:
RF = RandomForestRegressor()
RF.fit(X_train, Y_train)
print "The Random Forest Regressor Score is %0.6f" %RF.score(X_train,Y_train)

RF_pred = RF.predict(X_train)
print "Train data MSE is ",mean_squared_error(Y_train, RF_pred)

# =========================
# originally we're to predict with X_test and submit Y_test
# =========================
# RF_pred = RF.predict(X_test)

The Random Forest Regressor Score is 0.878687
Train data MSE is  0.0201127515181


In [32]:
print RF_pred.shape
print RF_pred

(1000000,)
[ 1.32948426  1.6915      1.54817143 ...,  2.02100833  1.88156675
  1.49672119]


In [33]:
RF_pred_test = RF.predict(X_test)

In [34]:
RF_pred_test.shape

(824230,)

In [35]:
write_to_file("RF-Grace-chemical.csv", RF_pred_test)

# Our own Models
## 1. LASSO

In [37]:
from sklearn import linear_model

alphas = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]

for alpha in alphas:
    print alpha
    clf = linear_model.Lasso(alpha=alpha)

    clf.fit(X_train, Y_train)
    print "The LASSO Score is %0.6f" %clf.score(X_train,Y_train)

    clf_pred = clf.predict(X_train)
    print "Train data MSE is ",mean_squared_error(Y_train, clf_pred)

0.001
The LASSO Score is 0.706164
Train data MSE is  0.0487157400752
0.01
The LASSO Score is 0.661220
Train data MSE is  0.0561671467747
0.1
The LASSO Score is 0.147332
Train data MSE is  0.141365964061
1.0
The LASSO Score is 0.000000
Train data MSE is  0.165792433263
10.0
The LASSO Score is 0.000000
Train data MSE is  0.165792433263
100.0
The LASSO Score is 0.000000
Train data MSE is  0.165792433263


In [38]:
clf = linear_model.Lasso(alpha=0.000001)

clf.fit(X_train, Y_train)
print "The LASSO Score is %0.6f" %clf.score(X_train,Y_train)

clf_pred = clf.predict(X_train)
print "Train data MSE is ",mean_squared_error(Y_train, clf_pred)

The LASSO Score is 0.710505
Train data MSE is  0.0479960181101


/Users/Grace/anaconda/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)


In [ ]:
print clf_pred.shape
print clf_pred

In [ ]:
clf_pred_test = clf.predict(X_test)

In [ ]:
clf_pred_test.shape

## 2. Ridge Regression

In [41]:
from sklearn.linear_model import Ridge

clf = linear_model.Ridge(alpha=1)

clf.fit(X_train, Y_train)
print "The Ridge Score is %0.6f" %clf.score(X_train,Y_train)

clf_pred = clf.predict(X_train)
print "Train data MSE is ",mean_squared_error(Y_train, clf_pred)

The Ridge Score is 0.710506
Train data MSE is  0.0479959372105
